# DataFrames: Read and Write Data
     
Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  In this example we read and write data with the popular CSV and Parquet formats, and discuss best practices when using these formats.

In [1]:
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/0eEsIA0O1iE?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/home/travis/miniconda/envs/test/lib/python3.7/site-packages/IPython/core/display.py:701: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [2]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

Client Scheduler: inproc://10.20.255.71/12172/1 Dashboard: http://10.20.255.71:8787/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [3]:
import dask
df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [4]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name);

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [5]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [6]:
!head data/2000-01-01.csv

timestamp,id,name,x,y
2000-01-01 00:00:00,989,Bob,0.2119961077100505,0.26757240622918443
2000-01-01 00:00:01,970,Ray,0.42415336472810883,0.7904127770173617
2000-01-01 00:00:02,995,Jerry,-0.8212670156020323,0.6735260291023122
2000-01-01 00:00:03,966,Bob,-0.621038752127522,0.2491328425640189
2000-01-01 00:00:04,1003,George,-0.5774353282134288,0.16410508560479364
2000-01-01 00:00:05,1048,Sarah,0.30454622481851046,-0.880542988380121
2000-01-01 00:00:06,1064,Edith,-0.4274683459759987,-0.1851934968073079
2000-01-01 00:00:07,1008,Alice,-0.42527671602983963,0.8609277180786004
2000-01-01 00:00:08,982,Xavier,0.796335093559374,-0.329009851535478


In [7]:
!head data/2000-01-30.csv

timestamp,id,name,x,y
2000-01-30 00:00:00,972,Wendy,0.37758568374599033,-0.9250702218333358
2000-01-30 00:00:01,959,Dan,0.20344883817820736,-0.4661827863750707
2000-01-30 00:00:02,1015,Kevin,0.7880079123153956,0.396586099131057
2000-01-30 00:00:03,994,Tim,-0.2674676787385706,0.9365052868266002
2000-01-30 00:00:04,963,George,-0.7679116716742616,-0.7839607514534053
2000-01-30 00:00:05,1003,Edith,0.38101478549718903,-0.6448740228665886
2000-01-30 00:00:06,956,Patricia,-0.7968974322079523,-0.039483223658898225
2000-01-30 00:00:07,952,Victor,-0.38778788928053354,-0.8431845731665792
2000-01-30 00:00:08,1053,Ingrid,0.15817674164156892,-0.4811113847328914


We can read one file with `pandas.read_csv` or many files with `dask.dataframe.read_csv`

In [8]:
import pandas as pd

df = pd.read_csv('data/2000-01-01.csv')
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,989,Bob,0.211996,0.267572
1,2000-01-01 00:00:01,970,Ray,0.424153,0.790413
2,2000-01-01 00:00:02,995,Jerry,-0.821267,0.673526
3,2000-01-01 00:00:03,966,Bob,-0.621039,0.249133
4,2000-01-01 00:00:04,1003,George,-0.577435,0.164105


In [9]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,object,int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [10]:
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,989,Bob,0.211996,0.267572
1,2000-01-01 00:00:01,970,Ray,0.424153,0.790413
2,2000-01-01 00:00:02,995,Jerry,-0.821267,0.673526
3,2000-01-01 00:00:03,966,Bob,-0.621039,0.249133
4,2000-01-01 00:00:04,1003,George,-0.577435,0.164105


## Tuning read_csv

The Pandas `read_csv` function has *many* options to help you parse files.  The Dask version uses the Pandas function internally, and so supports many of the same options.  You can use the `?` operator to see the full documentation string.

In [11]:
pd.read_csv?

In [12]:
dd.read_csv?

In this case we use the `parse_dates` keyword to parse the timestamp column to be a datetime.  This will make things more efficient in the future.  Notice that the dtype of the timestamp column has changed from `object` to `datetime64[ns]`.

In [13]:
df = dd.read_csv('data/2000-*-*.csv', parse_dates=['timestamp'])
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


## Do a simple computation

Whenever we operate on our dataframe we read through all of our CSV data so that we don't fill up RAM.  This is very efficient for memory use, but reading through all of the CSV files every time can be slow.

In [14]:
%time df.groupby('name').x.mean().compute()

CPU times: user 4.84 s, sys: 582 ms, total: 5.42 s
Wall time: 3.56 s


name
Alice       0.001320
Bob        -0.002526
Charlie    -0.000177
Dan         0.002305
Edith       0.001348
Frank       0.000589
George      0.000941
Hannah      0.000643
Ingrid     -0.001965
Jerry       0.004660
Kevin      -0.000837
Laura       0.003617
Michael     0.001669
Norbert    -0.001902
Oliver      0.002572
Patricia    0.000060
Quinn       0.000858
Ray         0.003332
Sarah      -0.003459
Tim         0.000614
Ursula      0.001423
Victor     -0.001401
Wendy       0.000639
Xavier      0.002753
Yvonne     -0.000053
Zelda      -0.000485
Name: x, dtype: float64

## Write to Parquet

Instead, we'll store our data in Parquet, a format that is more efficient for computers to read and write.

In [15]:
df.to_parquet('data/2000-01.parquet', engine='pyarrow')

In [16]:
!ls data/2000-01.parquet/

_common_metadata  part.15.parquet  part.22.parquet  part.2.parquet
_metadata	  part.16.parquet  part.23.parquet  part.3.parquet
part.0.parquet	  part.17.parquet  part.24.parquet  part.4.parquet
part.10.parquet   part.18.parquet  part.25.parquet  part.5.parquet
part.11.parquet   part.19.parquet  part.26.parquet  part.6.parquet
part.12.parquet   part.1.parquet   part.27.parquet  part.7.parquet
part.13.parquet   part.20.parquet  part.28.parquet  part.8.parquet
part.14.parquet   part.21.parquet  part.29.parquet  part.9.parquet


## Read from Parquet

In [17]:
df = dd.read_parquet('data/2000-01.parquet', engine='pyarrow')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [18]:
%time df.groupby('name').x.mean().compute()

CPU times: user 1.51 s, sys: 146 ms, total: 1.66 s
Wall time: 1.13 s


name
Alice       0.001320
Bob        -0.002526
Charlie    -0.000177
Dan         0.002305
Edith       0.001348
Frank       0.000589
George      0.000941
Hannah      0.000643
Ingrid     -0.001965
Jerry       0.004660
Kevin      -0.000837
Laura       0.003617
Michael     0.001669
Norbert    -0.001902
Oliver      0.002572
Patricia    0.000060
Quinn       0.000858
Ray         0.003332
Sarah      -0.003459
Tim         0.000614
Ursula      0.001423
Victor     -0.001401
Wendy       0.000639
Xavier      0.002753
Yvonne     -0.000053
Zelda      -0.000485
Name: x, dtype: float64

## Select only the columns that you plan to use

Parquet is a column-store, which means that it can efficiently pull out only a few columns from your dataset.  This is good because it helps to avoid unnecessary data loading.

In [19]:
%%time
df = dd.read_parquet('data/2000-01.parquet', columns=['name', 'x'], engine='pyarrow')
df.groupby('name').x.mean().compute()

CPU times: user 1.35 s, sys: 86.3 ms, total: 1.43 s
Wall time: 1.01 s


name
Alice       0.001320
Bob        -0.002526
Charlie    -0.000177
Dan         0.002305
Edith       0.001348
Frank       0.000589
George      0.000941
Hannah      0.000643
Ingrid     -0.001965
Jerry       0.004660
Kevin      -0.000837
Laura       0.003617
Michael     0.001669
Norbert    -0.001902
Oliver      0.002572
Patricia    0.000060
Quinn       0.000858
Ray         0.003332
Sarah      -0.003459
Tim         0.000614
Ursula      0.001423
Victor     -0.001401
Wendy       0.000639
Xavier      0.002753
Yvonne     -0.000053
Zelda      -0.000485
Name: x, dtype: float64

Here the difference is not that large, but with larger datasets this can save a great deal of time.

## Learn more

http://dask.pydata.org/en/latest/dataframe-create.html